In [1]:
import sys
sys.path.append('/workspaces/secure_inference/')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb

from research.distortion.parameters.classification.resent.resnet18_8xb16_cifar100 import Params as resnet18_8xb16_cifar100_Params

In [15]:
run_ids = {
    "layer4_1_2_best_features_False": "bazo8rfa",
    "layer4_1_2_best_features_True": "bfc5rag7",
    "layer4_1_1_best_features_False": "g9wu3f0l",
    "layer4_1_1_best_features_True": "wgeeta8n",
    "layer4_0_2_best_features_False": "48mec68i",
    "layer4_0_2_best_features_True": "6itprzbc",
    "layer4_0_1_best_features_False": "npbnf6gj",
    "layer4_0_1_best_features_True": "b3omk9uc",
    "layer3_1_2_best_features_False": "n8y3felm",
    "layer3_1_2_best_features_True": "0itdd8c7",
    "layer3_1_1_best_features_False": "q9q3e3sc",
    "layer3_1_1_best_features_True": "7xrw3awj",
    "layer3_0_2_best_features_False": "vmlm2ke0",
    "layer3_0_2_best_features_True": "elk4tjuo",
    "layer3_0_1_best_features_False": "7hdg5s9c",
    "layer3_0_1_best_features_True": "6ipsgqgy",
    "layer2_1_2_best_features_False": "ofdijq57",
    "layer2_1_2_best_features_True": "36sxfmh8",
    "layer2_1_1_best_features_False": "sebziinp",
    "layer1_1_2_best_features_True": "w4g2zuh8",
    "layer1_1_1_best_features_False": "s5cafyfn",
    "layer1_1_1_best_features_True": "d0fv0atd",
    "layer1_0_2_best_features_False": "uzb7l0q6",
    "layer1_0_2_best_features_True": "37bp0ys9",
    "layer1_0_1_best_features_False": "xowe5s6r",
    "layer1_0_1_best_features_True": "65vor0e3",
    "stem_best_features_False": "r51njdxv",
    "stem_best_features_True": "wrcpu584",
    "layer2_0_2_best_features_False": "0d7udjig",
    "layer2_0_2_best_features_True": "570osv9r",
    "layer2_0_1_best_features_False": "34iwzv0b",
    "layer2_0_1_best_features_True": "d72hx7ez",
    "layer1_1_2_best_features_False": "twx66r0i",
    "layer2_1_1_best_features_True": "rc0a4o99",
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
run = api.run("yoni-bornfeld/private_inference/xowe5s6r")

In [8]:
history = run.history()
history["val/accuracy_top-1"].unique()

array([        nan, 77.50999451, 77.25      , 77.40999603, 77.40000153,
       77.41999817, 77.43999481, 77.37999725, 77.38999939])

In [ ]:
api = wandb.Api()

In [32]:
layers_names = resnet18_8xb16_cifar100_Params().LAYER_NAMES
total = []
for layer_name in layers_names:
    print(layer_name)
    layer_dict = {}
    for use in [True, False]:
        run_name = f"{layer_name}_best_features_{use}"
        run_id = run_ids[run_name]
        run = api.run(f"yoni-bornfeld/private_inference/{run_id}")
        summary = run.summary
        last_dict = {f"{use}_{k}".replace(f"{layer_name}/", ""): v for k, v in summary.items()}
        layer_dict.update(last_dict)
    layer_dict["layer_name"] = layer_name
    total.append(layer_dict)

total = pd.DataFrame(total)

stem
layer1_0_1
layer1_0_2
layer1_1_1
layer1_1_2
layer2_0_1
layer2_0_2
layer2_1_1
layer2_1_2
layer3_0_1
layer3_0_2
layer3_1_1
layer3_1_2
layer4_0_1
layer4_0_2
layer4_1_1
layer4_1_2


In [44]:
total.columns

Index(['True__runtime', 'True__step', 'True__timestamp', 'True_learning_rate',
       'True_momentum', 'True_all_zero_ratio', 'True_batch_index',
       'True_cluster_amount_mean', 'True_cluster_amount_sum',
       'True_decisions_accuracy', 'True_decisions_id_accuracy',
       'True_decisions_zero_accuracy', 'True_failed_ratio',
       'True_id_channel_ratio', 'True_max_inter', 'True_min_inter',
       'True_sigmoid_factor', 'True_use_sigmoid', 'True_total/all_zero_ratio',
       'True_total/drelu_ratio', 'True_total/fail_ratio',
       'True_total/id_channel_ratio', 'True_train/loss',
       'True_val/accuracy_top-1', 'True_val/accuracy_top-5', 'False__runtime',
       'False__step', 'False__timestamp', 'False_learning_rate',
       'False_momentum', 'False_all_zero_ratio', 'False_batch_index',
       'False_cluster_amount_mean', 'False_cluster_amount_sum',
       'False_decisions_accuracy', 'False_decisions_id_accuracy',
       'False_decisions_zero_accuracy', 'False_failed_ratio',


In [46]:
total['acc1_diff'] = total["True_val/accuracy_top-1"] - total["False_val/accuracy_top-1"]
total[['layer_name', 'acc1_diff', 'True_val/accuracy_top-1', 'False_val/accuracy_top-1', 'True_total/drelu_ratio']]

,layer_name,acc1_diff,True_val/accuracy_top-1,False_val/accuracy_top-1,True_total/drelu_ratio
0,stem,0.110001,77.699997,77.589996,0.014648
1,layer1_0_1,0.330002,77.720001,77.389999,0.028671
2,layer1_0_2,-0.029999,77.449997,77.479996,0.022461
3,layer1_1_1,-0.049995,77.389999,77.439995,0.032349
4,layer1_1_2,-0.040001,77.709999,77.750000,0.012573
5,layer2_0_1,0.590004,76.750000,76.159996,0.176910
6,layer2_0_2,0.050003,77.419998,77.369995,0.082581
7,layer2_1_1,0.139999,77.549995,77.409996,0.076172
8,layer2_1_2,0.519997,77.419998,76.900002,0.126007
9,layer3_0_1,0.590004,76.930000,76.339996,0.343689


In [47]:
np.mean(77.5 - total["True_val/accuracy_top-1"])

0.17000175924862132

In [31]:
dict(run.summary)

{'_runtime': 1112.270267009735,
 '_step': 4073,
 '_timestamp': 1733002808.470412,
 'learning_rate': 0.0004,
 'momentum': 0.9,
 'stem/all_zero_ratio': 0,
 'stem/batch_index': 4073,
 'stem/cluster_amount_mean': 96,
 'stem/cluster_amount_sum': 960,
 'stem/decisions_accuracy': 0.9411264038085936,
 'stem/decisions_id_accuracy': 0.9268582929602616,
 'stem/decisions_zero_accuracy': 0.8401338389510619,
 'stem/failed_ratio': 0,
 'stem/id_channel_ratio': 0.84375,
 'stem/max_inter': 1,
 'stem/min_inter': 1,
 'stem/sigmoid_factor': None,
 'stem/use_sigmoid': 0,
 'total/all_zero_ratio': 0,
 'total/drelu_ratio': 0.0146484375,
 'total/fail_ratio': 0,
 'total/id_channel_ratio': 0.84375,
 'train/loss': 0.01527455858886242,
 'val/accuracy_top-1': 77.69999694824219,
 'val/accuracy_top-5': 94.1500015258789}

In [14]:
dict(history.loc[history["_step"].idxmax()])

{'layer1_0_1/min_inter': 1.0,
 '_step': 6000.0,
 'layer1_0_1/max_inter': 1.0,
 '_runtime': 1510.88352060318,
 'layer1_0_1/use_sigmoid': 0.0,
 '_timestamp': 1733006988.1413455,
 'layer1_0_1/batch_index': 6000.0,
 'momentum': 0.9,
 'learning_rate': 0.0004,
 'train/loss': 0.015430265478789807,
 'layer1_0_1/cluster_amount_sum': 1867.0,
 'layer1_0_1/id_channel_ratio': 0.625,
 'total/fail_ratio': 0.0,
 'total/drelu_ratio': 0.0284881591796875,
 'layer1_0_1/decisions_zero_accuracy': 0.8190497584587462,
 'total/all_zero_ratio': 0.0,
 'layer1_0_1/cluster_amount_mean': 77.79166666666667,
 'total/id_channel_ratio': 0.625,
 'layer1_0_1/decisions_accuracy': 0.9104395548502605,
 'layer1_0_1/all_zero_ratio': 0.0,
 'layer1_0_1/decisions_id_accuracy': 0.8154718100154422,
 'layer1_0_1/failed_ratio': 0.0,
 'val/accuracy_top-1': 77.38999938964844,
 'val/accuracy_top-5': 94.16999816894531}

In [9]:
len(resnet18_8xb16_cifar100_Params().LAYER_NAMES)

17

In [ ]:
stats_dir = ""
stats = {}
for layer_name in resnet18_8xb16_cifar100_Params:
    pass